# Transfer learning sur les images : MobileNet


* import des librairies

In [1]:
# import des librairies

%matplotlib inline
import matplotlib.pyplot as plt
import pickle as pk
import pandas as pd
import numpy as np
import os, time

os.environ['KMP_DUPLICATE_LIB_OK']='True'
from PIL import Image
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,Dropout, Flatten, Dense, Activation, Input
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from timeit import default_timer as timer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback, ModelCheckpoint, TerminateOnNaN,LearningRateScheduler,TensorBoard

from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder # one-hot encoding for age
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.applications import MobileNetV2, EfficientNetB7,ResNet152V2, Xception

from random import sample
import shutil


#import cv2 #import OpenCV -> pip install opencv-python
import numpy as np
import glob

In [2]:
import os #Miscellaneous operating system interfaces
#https://docs.python.org/3/library/os.html
#get current working directory
path = os.getcwd() + 'C:\\Users\\admin\\Desktop\\Data challenge\\train_input\\train_input\\images'
path

'C:\\Users\\admin\\Desktop\\Data challenge\\train_input\\train_input\\moco_featuresC:\\Users\\admin\\Desktop\\Data challenge\\train_input\\train_input\\images'

* Récuperer les données: pre process

In [3]:
df = pd.read_csv("train_output_76GDcgx.csv", index_col=0)

df.head()

,Target
Sample ID,
ID_001.npy,0
ID_002.npy,1
ID_005.npy,0
ID_006.npy,0
ID_007.npy,1


In [4]:
df['image_path'] = path + df['image']

* Separation des données :

In [5]:
data = df[['image','prdtypecode']]
target = df['prdtypecode']

In [6]:
X_train_img, X_test_img, y_train, y_test = train_test_split(data, target,train_size=0.1,test_size=0.2, random_state=123)


In [7]:
X_train_img

,image,prdtypecode
ID,,
48657,image_1151986340_product_2590702548.jpg,2403
22255,image_1104460778_product_1832787442.jpg,1280
51900,image_1260356330_product_3893244250.jpg,1302
51514,image_851402132_product_83232680.jpg,1160
62795,image_1287037647_product_58582799.jpg,10
...,...,...
27830,image_1314428192_product_4202302602.jpg,2585
27986,image_1319608338_product_4214628106.jpg,2060
24537,image_1227217601_product_102678621.jpg,10


* Transformation des données:

In [8]:
X_train_img['prdtypecode']=X_train_img['prdtypecode'].astype(str)
X_test_img['prdtypecode']=X_test_img['prdtypecode'].astype(str)

In [9]:
X_train_img.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7485 entries, 48657 to 7106
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        7485 non-null   object
 1   prdtypecode  7485 non-null   object
dtypes: object(2)
memory usage: 175.4+ KB


In [10]:
from keras.applications.efficientnet import preprocess_input
import tensorflow as tf

batch = 32


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                preprocessing_function = preprocess_input,
                                   shear_range = 0.5,
                                   zoom_range = 0.1,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                    brightness_range = [0.9,1.1],
                                   fill_mode='nearest')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                              preprocessing_function = preprocess_input
                                                              )

train_set = train_datagen.flow_from_dataframe(dataframe=X_train_img,
                                              directory=path,
                                              x_col = "image",
                                              y_col = "prdtypecode",
                                              class_mode ="sparse",
                                              target_size = (224, 224),
                                              batch_size = batch)

test_set = test_datagen.flow_from_dataframe(dataframe=X_test_img,
                                              directory=path,
                                              x_col = "image",
                                              y_col = "prdtypecode",
                                            class_mode ="sparse",
                                              target_size = (224, 224),
                                              batch_size = batch,
                                           shuffle=False)

Found 7485 validated image filenames belonging to 27 classes.
Found 14971 validated image filenames belonging to 27 classes.


In [ ]:
#APPLY SOME TRANSFORMATIONS TO DATA
from keras.applications.vgg16 import preprocess_input
import tensorflow as tf


batch = 64


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                preprocessing_function = None,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                              preprocessing_function = preprocess_input
                                                         )



train_set = train_datagen.flow_from_dataframe(dataframe=X_train_img,
                                              directory=path,
                                              x_col = "image",
                                              y_col = 'prdtypecode',
                                              seed=42,
                                              class_mode="categorical",
                                              target_size = (100, 100),
                                              batch_size = batch,
                                             shuffle=True)

test_set = test_datagen.flow_from_dataframe(dataframe=X_test_img,
                                              directory=path,
                                              x_col = "image",
                                              y_col = "prdtypecode",
                                              class_mode="categorical",
                                              seed=42,
                                              target_size = (100, 100),
                                              batch_size = batch,
                                           shuffle=False)

# Callbacks :

In [11]:
from tensorflow.keras import callbacks

model_type = 'Functional'
existing_model = 'efficientNetB1'
version = 'v3'
filename = model_type + '_' + existing_model + '_' +  version
model_path = 'models_output\\' + existing_model + '\\'

early_stopping = callbacks.EarlyStopping(monitor='val_accuracy',
                                         patience=2,
                                         mode='max',
                                         restore_best_weights=True)


checkpoint = callbacks.ModelCheckpoint(filepath= model_path + filename + '.hdf5', 
                                       monitor='val_accuracy',
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='max',
                                       save_freq='epoch')

#  IMPLEMENTATION DU MODELE :efficientNetB1

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import keras
import keras.applications as kapps
import tensorflow as tf

In [12]:
from keras.layers import BatchNormalization

basemodel = tf.keras.applications.EfficientNetB1(input_shape = (254, 254, 3),
                                              include_top = False,
                                              # drop_connect_rate=0.4,
                                              weights = 'imagenet')



model = Sequential()

model.add(basemodel) 

model.add(GlobalAveragePooling2D()) 

model.add(Dense(1024,activation='relu'))

model.add(BatchNormalization(trainable = True,axis=1))

model.add(Dropout(rate=0.5))

model.add(Dense(512, activation='relu'))

model.add(BatchNormalization(trainable = True,axis=1))


model.add(Dense(27, activation='softmax'))

In [ ]:
unfreezed_layers = 10 #Nombre de couches a décongeler pour aplique le finetuning: Voir livre Deep Learning with python

# Freezer les couches du efficientNetB1
for layer in model.layers: 
    layer.trainable = False

# DeFreezer les quelques couches du VGG16
for layer in model.layers[-unfreezed_layers:]: 
    layer.trainable = True
    
model.summary()

In [13]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [14]:
mobile_model_history = model.fit_generator(train_set, 
                                                  steps_per_epoch=int(len(y_train.values)/batch),
                                                  validation_data = test_set, 
                                                  validation_steps = int(len(y_test)/batch),
                                                  epochs=10, 
                                                  workers=-1,
                                                  callbacks=[early_stopping, checkpoint],
                                                  verbose = True)

C:\Users\ridf\AppData\Local\Temp\ipykernel_8332\3968565770.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  mobile_model_history = model.fit_generator(train_set,


Epoch 1/10
233/233 [==============================] - ETA: 0s - loss: 2.6970 - accuracy: 0.2972

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

* Courbe d'évolution model par epoch

In [ ]:
import matplotlib.pyplot as plt 

train_acc = mobile_model_history.history['accuracy']
val_acc = mobile_model_history.history['val_accuracy']

plt.figure(figsize=(12,4))

plt.subplot(121)
plt.plot(mobile_model_history.history['loss'])
plt.plot(mobile_model_history.history['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')

plt.subplot(122)
plt.plot(mobile_model_history.history['accuracy'])
plt.plot(mobile_model_history.history['val_accuracy'])
plt.title('Model acc by epoch')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')
plt.show()


## Matrice de confusion , Evaluation et Erreurs :

In [ ]:
y_predmobile = mobile_model.predict(test_set)

In [ ]:
fit_labels = dict((v,k) for k,v in (train_set.class_indices).items())

y_predmobile = [fit_labels[i] for i in np.argmax(y_predmobile, axis=1)]

y_test = y_test.tolist()

In [ ]:
y_pred1 = []
for element in y_predmobile:
    y_pred1.append(int(element))
    
y_predmobile=y_pred1

In [ ]:
from sklearn.metrics import f1_score, classification_report

print(classification_report(y_test,y_predmobile))

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline
import itertools # Pour créer des iterateurs


#Réponse valable:
cnf_matrix = metrics.confusion_matrix(y_test,y_predmobile,labels=list(set(y_predmobile)))

pond_matrix = []
for line in cnf_matrix:
    pond_line = []
    for cell in line:
        pond_line.append(round(cell/sum(line),2))
    pond_matrix.append(pond_line)
    #print(sum(line))
    #print(sum(pond_line))
cnf_matrix = np.array(pond_matrix)

###Optionnel: Afficher une matrice de confusion sous forme de tableau coloré
#classes = range(0,26)
classes = set(y_predmobile)

plt.figure(figsize=(17,17))

plt.imshow(cnf_matrix, interpolation='nearest',cmap='Blues')
plt.title("Matrice de confusion")
plt.colorbar()
tick_marks = np.arange(len(classes))
#tick_marks = set(y_test)
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
    plt.text(j, i, cnf_matrix[i, j],
             horizontalalignment="center",
             color="white" if cnf_matrix[i, j] > ( cnf_matrix.max() / 2) else "black")

plt.ylabel('Vrais labels')
plt.xlabel('Labels prédits')
plt.show()

# Erreurs :

In [ ]:
import cv2 #import OpenCV
import matplotlib.pyplot as plt # Pour l'affichage d'images
import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline

classes = {'10':' Livres, Couvertures de livres ',
           '40':' Jeux videos, CDs + mais aussi equipements, cables, etc. ',
           '50':' Jeux Vidéos, Equipements ',
           '60':' Consoles, Manettes, croix, boutons, ecrans ',
           '1140':' Figurines, Personnages et objets, parfois dans des boites ',
           '1160':' Cartes, Rectangles, beaucoup de couleurs ',
           '1180':' Figurines et boites ',
           '1280':' Jouets, poupées nounours, equipements enfants',
           '1281':' Jeux enfants, Boites et autres, couleurs flashy',
           '1300':' Jeux techniques, Equipement, petites machines ',
           '1301':' Habits bébés, petites photos ',
           '1302':' Equipements, Habits, outils, jouets, objets sur fond blanc',
           '1320':' Matériel et meubles bébé poussettes, habits',
           '1560':' Meubles, matelas canapés lampes, chaises',
           '1920':' Oreillers, coussins, draps',
           '1940':' Alimentations, conserves boites d gateaux',
           '2060':' Décorations',
           '2220':' Equipements divers pour animaux',
           '2280':' Livres et revues anciennes',
           '2403':' Livres et revues de collection',
           '2462':' Equipement jeux, play stations',
           '2522':' Cahiers, carnets, marque pages',
           '2582':' Matériel, meubles et outils pour le jardin',
           '2583':' Equipements technique pour la maison et exterieur (piscines), produits',
           '2585':' Idem 2583:  Equipements technique pour la maison et exterieur (piscines), produits',
           '2705':' Livres',
           '2905':' Jeux vidéos'}


error_indexes = []
for i in range(len(y_predmobile)):
    if (y_predmobile[i] != y_test[i]):
        error_indexes += [i]

images = 8
        
j = 1

plt.figure(figsize = (20,20))
#plt.subplot(10,5,1)
plt.axis('off')

for i in np.random.choice(error_indexes, size = images):
    img = cv2.imread(path + '\\' + X_test_img.iloc[i]['image'])
    img = cv2.resize(img, (240, 240), interpolation=cv2.INTER_CUBIC)
    #img = cv2.imread(path + '\\' + X_test_img.iloc[0]['image_name'], cv2.IMREAD_COLOR)
    #img = img.reshape(28, 28)
    
    #plt.figure(figsize=(5,10))
    
    plt.subplot(4,2,j)
    
    #plt.subplot(5, 5, j)
    j = j + 1
    plt.axis('off')
    plt.imshow(img, cmap=cm.binary, interpolation='None')
    plt.title('True Label: ' + classes[str(y_test[i])] \
              + '\n' + 'Prediction: '+ classes[str(y_predmobile[i])] \
              + '\n' + 'Confidence: '+ str(round(np.max(y_predmobile, axis=0),2)))

In [ ]:
mobile_model.save('efficientNEtB1.h5')